<a href="https://colab.research.google.com/github/jetshaggy85/info5731/blob/master/Fellers_selfstudy12_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corrected TensorFlow/Keras **Version**

In [0]:
!wget https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv

--2019-04-18 02:41:36--  https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘customer_churn.csv’

customer_churn.csv  100%[===================>] 668.81K  --.-KB/s    in 0.04s   

2019-04-18 02:41:41 (17.6 MB/s) - ‘customer_churn.csv’ saved [684858/684858]



In [0]:
import pandas as pd

In [0]:
df=pd.read_csv('customer_churn.csv')

In [0]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
!pip install -q tensorflow==2.0.0-alpha0

    100% |████████████████████████████████| 79.9MB 361kB/s 
    100% |████████████████████████████████| 61kB 27.0MB/s 
    100% |████████████████████████████████| 419kB 22.8MB/s 
    100% |████████████████████████████████| 3.0MB 12.4MB/s 


In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
numeric_columns=['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']
categorical_columns=['Geography','Gender','HasCrCard','IsActiveMember']

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train, test = train_test_split(df,test_size=.2,random_state=1)

In [0]:
train, valid = train_test_split(train,test_size=.2,random_state=1)

In [0]:
from tensorflow import feature_column

In [0]:
tf.random.set_seed(1)

In [0]:
feature_columns=[]

In [0]:
for header in numeric_columns:
   feature_columns.append(feature_column.numeric_column(header))

In [0]:
def get_one_hot_rom_categorical(colname):
  categorical=feature_column.categorical_column_with_vocabulary_list(
  colname,
  train[colname].unique())
  return feature_column.indicator_column(categorical)

In [0]:
for col in categorical_columns:
  feature_columns.append(get_one_hot_rom_categorical(col))

In [0]:
feature_columns

[NumericColumn(key='CreditScore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Tenure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Balance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NumOfProducts', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EstimatedSalary', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Geography', vocabulary_list=('France', 'Spain', 'Germany'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Gender', vocabulary_list=('Male', 'Female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(c

In [0]:
from tensorflow.keras import layers

In [0]:
feature_layer=layers.DenseFeatures(feature_columns)

In [0]:
model=keras.Sequential()

In [0]:
model.add(feature_layer)
model.add(layers.Dense(200,activation='relu'))
model.add(layers.Dense(100,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [0]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [0]:
def df_to_tfdata(df, shuffle=True, bs=32):
  df = df.copy()
  labels = df.pop('Exited')
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df), seed=1)
  ds = ds.batch(bs)
  return ds

In [0]:
train_ds=df_to_tfdata(train)
valid_ds = df_to_tfdata(valid, shuffle=False)
test_ds = df_to_tfdata(test, shuffle=False)

In [0]:
model.fit(train_ds,
         validation_data = valid_ds,
         epochs=5)

Epoch 1/5
200/200 [==============================] - 5s 23ms/step - loss: 0.4289 - accuracy: 0.8058 - val_loss: 0.4044 - val_accuracy: 0.8306
Epoch 2/5
200/200 [==============================] - 4s 21ms/step - loss: 0.3750 - accuracy: 0.8382 - val_loss: 0.3802 - val_accuracy: 0.8419
Epoch 3/5
200/200 [==============================] - 4s 22ms/step - loss: 0.3568 - accuracy: 0.8470 - val_loss: 0.3765 - val_accuracy: 0.8413
Epoch 4/5
200/200 [==============================] - 4s 21ms/step - loss: 0.3491 - accuracy: 0.8503 - val_loss: 0.3742 - val_accuracy: 0.8450
Epoch 5/5
200/200 [==============================] - 4s 21ms/step - loss: 0.3438 - accuracy: 0.8554 - val_loss: 0.3740 - val_accuracy: 0.8444


In [0]:
preds=model.predict(test_ds)

In [0]:
import numpy as np

labels = (preds > 0.5).astype(np.int)
print(labels)

[[0]
 [0]
 [0]
 [0]
 ...
 [1]
 [0]
 [0]
 [0]]


In [0]:
from sklearn import metrics

In [0]:
ytest= test.Exited.values

In [0]:
print(metrics.classification_report(ytest, labels))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92      1585
           1       0.72      0.57      0.64       415

   micro avg       0.87      0.87      0.87      2000
   macro avg       0.81      0.76      0.78      2000
weighted avg       0.86      0.87      0.86      2000



In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
confusion_matrix(ytest, labels)

array([[1495,   90],
       [ 178,  237]])